In [2]:
# SQL Alchemy - cria database de todos os bancos de dados

# psycopg2 - apenas de Postgres

import psycopg2 as pg2
import pandas as pd

#nunca coloque suas credenciais no notebook: usar ginger?
#credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
user = 'member'
pwd = 'cdspa'

In [3]:
#conexão com o banco de dados
conn = pg2.connect( 
                    user = user,
                    password = pwd,
                    host = host,
                    port = port,
                    database = database
            )

In [14]:
#psycopg precisa de um cursor e .execute, ele não retorna dados
cursor = conn.cursor()

query_schema =  """
                SELECT nspname
                FROM pg_catalog.pg_namespace 
                
               """
cursor.execute( query_schema )

#para trazer os dados precisamos do fecth all
record = cursor.fetchall()
#sempre fechar a conexão do banco de dados pois há limite de portas
cursor.close()
conn.close()

In [12]:
cursor = conn.cursor()

query_tables =  """
                SELECT tablename
                FROM pg_tables
                WHERE schemaname = 'pa004'
                
               """
cursor.execute( query_tables )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


In [19]:
cursor = conn.cursor()

query_table_users =  """
                SELECT *
                FROM pa004.users u
                WHERE u.age > 44 
                limit 10
                
               """
cursor.execute( query_table_users )

record = cursor.fetchall()
print( record )

cursor.close()
conn.close()

InterfaceError: connection already closed

In [20]:
pd.DataFrame( record )

,0,1,2,3,4
0,2,Male,76,3.0,26.0
1,3,Male,47,28.0,26.0
2,8,Female,56,28.0,26.0
3,11,Female,47,35.0,124.0
4,14,Male,76,28.0,13.0
5,15,Male,71,28.0,30.0
6,20,Female,60,33.0,124.0
7,21,Male,65,28.0,124.0
8,22,Male,49,28.0,124.0
9,27,Female,51,28.0,124.0


In [7]:
import pandas.io.sql as psql

query_table_users =  """
                SELECT *
                FROM pa004.users u LEFT JOIN pa004.vehicle v ON ( u.id = v.id )
                                   LEFT JOIN pa004.insurance i ON ( u.id = i.id )
                WHERE u.age > 44 
                                
               """
df = psql.read_sql( query_table_users, conn )

In [13]:
#este é o método mais usado
df1 = pd.read_sql( query_table_users, conn )
df1 = df1.drop( df1.columns[[5, 9]], axis=1 )
df1.head()

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,56,28.0,26.0,1,1-2 Year,Yes,0,32031.0,72,1
1,Female,47,35.0,124.0,1,1-2 Year,Yes,0,47576.0,46,1
2,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
3,Female,54,28.0,122.0,1,1-2 Year,No,0,38560.0,184,0
4,Female,49,3.0,152.0,1,1-2 Year,No,1,25553.0,130,0
